In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import math

In [2]:
os.chdir("/Users/Ellen/Documents/Dokumenter/DTU/7. Semester/Bachelor/CSV files")

poiNames = pd.read_csv("POInames.csv")
dfProb = pd.read_csv("Biprob.csv")

In [3]:
poiStop = poiNames['StopName'].tolist()
stop = ['home','work','other']

#List with all stops
stop.extend(poiStop)

In [4]:
#probability dataframe to matrix
p = dfProb.to_numpy()

In [5]:
#selected stops
stopSelect = ['home', 'work', 'other', 'School', 'Restaurant', 'University', 'Bar', 'Train Station']

In [6]:
#stopIndex of the 10 most probable stops from home
stopIndex = dfProb.sum(axis=0).nlargest(10).index

In [7]:
#name of the 10 most probable stops from home
largeStop = []

for i in range(0,len(stopIndex)):
    largeStop.append(stop[int(stopIndex[i])])

In [8]:
#merge the two lists with the selected stops and the most probable stops
stopSelect.extend(largeStop)

In [9]:
#unique stops from the selected and most probable stops. 
#this is the stops that will be used to the clustering
StopList = list(set(stopSelect))

In [10]:
#the index of the stops that should be used for the clustering
StopIndex = []
for i in range(0, len(StopList)):
    StopIndex.append(stop.index(StopList[i]))

In [11]:
#probability matrix with all users, and the selected stop types
pSelect = p[:,StopIndex]

In [12]:
#pselect to dataframe and save as csv
pd.DataFrame(pSelect).to_csv('pSelect.csv', index = False)

In [13]:
#StopList(the names) to dataframe and save as csv
pd.DataFrame(StopList).rename(columns={0: 'Stop'}).to_csv('StopList.csv', index = False)

In [14]:
#set seed
# Gap Statistic for K means
def optimalK(data, nrefs, maxClusters):
    """
    Calculates KMeans optimal K using Gap Statistic 
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters+1)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[], 's_k':[], 'gap-s_k':[]})
    for gap_index, k in enumerate(range(1, maxClusters+1)):

# Holder for reference dispersion results
        refDisps = np.zeros(nrefs)

# For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):
            
            # Create new random reference set
            randomReference = np.zeros(np.shape(data))
            for j in range (0,np.shape(data)[1]):
                low = min(data[:,j])
                high = max(data[:,j])
                randomReference[:,j] = np.random.uniform(low, high, size=len(data))        
            
            # Fit to it
            km = KMeans(k)
            km.fit(randomReference)
            
            #refDisp 
            refDisps[i] = km.inertia_
              

# Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)
        
        origDisp = km.inertia_
# Calculate gap statistic
        gap = (1/nrefs)*np.sum(np.log(refDisps)) - np.log(origDisp)
        l =(1/nrefs)*np.sum(np.log(refDisps))
        sd_k = ((1/nrefs)*np.sum((np.log(refDisps)-l)**2))**(1/2)
        s_k = sd_k*math.sqrt(1+1/nrefs)
        print(k, " Gap: ", gap, " s_k ", s_k)
        
# Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
    
        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap, 's_k':s_k, 'gap-s_k':gap-s_k, 'wss':origDisp}, ignore_index=True)
    return (gaps.argmax() + 1, resultsdf)

In [15]:
score_g, df = optimalK(pSelect, nrefs=20, maxClusters=60)

1  Gap:  1.9274053165602503  s_k  0.0011946881817621912
2  Gap:  2.324277060949047  s_k  0.0009444814646489825
3  Gap:  2.5625902200618977  s_k  0.0007956512885448796
4  Gap:  2.6706754193463826  s_k  0.0008824780471871749
5  Gap:  2.7514022761462584  s_k  0.0007345819389902171
6  Gap:  2.811932662164086  s_k  0.0007037514363361306
7  Gap:  2.8665358203453284  s_k  0.0007014448941576204
8  Gap:  2.9088101969829667  s_k  0.0009277160817020997
9  Gap:  2.9360487486334055  s_k  0.0006056032667683121
10  Gap:  2.9544526151847057  s_k  0.001056794460244449
11  Gap:  2.9794520083646523  s_k  0.0008801211466050075
12  Gap:  3.000461041259147  s_k  0.0008003232800707165
13  Gap:  3.019604756616351  s_k  0.0008664534268694027
14  Gap:  3.0370675336592967  s_k  0.0006627837082962976
15  Gap:  3.0531840690777106  s_k  0.0008459756549141588
16  Gap:  3.0660575958919356  s_k  0.0008498907754576768
17  Gap:  3.0774998600138366  s_k  0.0008085684415643222
18  Gap:  3.0901532379518306  s_k  0.00073951

In [16]:
np.savetxt('gapSelect.csv', df, delimiter=",")

In [17]:
#find the number of clusters, where gap(k) >= gap(k+1)-s_k
#gap1
cluster = []
for i in range(df.shape[0]-1):
    if df['gap'][i] >= df['gap-s_k'][i+1]:
        #cluster.append(i)
        cluster.append(i+1)
        
cluster

[50, 55]